In [315]:
import jp_util.common_path as cp
from jp_util.pandas_util import rd_csv_sig, to_csv_sig
from jp_util.file_util import get_obj_by_pickle_path, export_to_pickle_path
from jp_util.common_util import l, p
import jp_util.jap_base_class as jp
from importlib import reload
import pandas as pd
import numpy as np
import os
import re

reload(cp)

<module 'jp_util.common_path' from 'D:\\02.work_project\\jp_util\\src\\jp_util\\common_path.py'>

In [316]:
# 获取13万的基础词条
df_base_word_130000 = jp.JapBaseWordClass.gen_df_base_word()
# df_base_word
p(len(df_base_word_130000))

133637


## 词频调整处理步骤
### 一. 标注出需要删除的词和哪些是高中词汇 
1. 整理2400_v3_raw.csv的词汇， 在原有的基础词条中标注，哪些是高中词汇。共有2648个高中词汇df_merge_base_2400_v3.csv
2. 删除13万词汇中的(含特殊字符的词汇，数字，字母，特殊字符等) 540个 special_char_in_base_word.csv 
3. 删除13万词汇中的 中文词汇  2567个  单词发音(纯中文无pron).xlsx
4. 删除徐老师标注的 24个  异常词（'为'出现在中文词中)，所以比徐老师的少1个
5. 删除徐老师标注的 66个  特殊名词
6. 删除徐老师标注的 176个  语法类
7. 删除徐老师标注的 34个  感叹词
### 二. 从高中2400, 5000篇阅读的词汇，长词汇，小学馆中将标注需要删除的词删除
1. 

In [317]:
# 1. 整理2400_v3_raw.csv的词汇， 在原有的基础词条中标注，哪些是高中词汇。
df_2400_v3 = rd_csv_sig(cp.r_word_2400_v3_csv)
p(len(df_2400_v3))

# 把2400_v3去重
df_2400_v3_drop_dup = df_2400_v3.drop_duplicates(subset=["spell"], keep="first")
p(len(df_2400_v3_drop_dup))

df_merge_base_2400_v3 = pd.merge(
    df_base_word_130000,
    df_2400_v3_drop_dup[["spell"]],
    left_on="spell",
    right_on="spell",
    how="left",
    indicator="ck_2400_v3",
)
# 标注高中高中词汇，用于后续调整词频
df_merge_base_2400_v3["is_for_student"] = np.where(
    (df_merge_base_2400_v3["spell_type"] == 1)
    | (df_merge_base_2400_v3["ck_2400_v3"] == "both"),
    1,
    0,
)

to_csv_sig(df_merge_base_2400_v3, "d:/tmp/df_merge_base_2400_v3.csv")
l("高中词汇标注完毕ok")

2682
2624
----------------------------------------高中词汇标注完毕ok----------------------------------------


In [318]:
# 包含特殊字符的词(数字，符号，英文)
filtered_df_freq = df_base_word_130000[
    df_base_word_130000["spell"].str.contains(cp.filter_pattern, na=False)
].copy()
to_csv_sig(filtered_df_freq, 'd:/tmp/special_char_in_base_word.csv')
l("ok")

----------------------------------------ok----------------------------------------


In [319]:
### 二. 从高中2400, 5000篇阅读的词汇，长词汇，小学馆中将标注需要删除的词删除
to_del = rd_csv_sig("D:/Dropbox/06.wanjuan/01.设计/99.基础词汇调整/to_del.csv")
p(len(to_del))

# 1.从高中2400中删除 需要删除的词
df_2400_v3_drop_dup_filtered = df_2400_v3_drop_dup[
    ~df_2400_v3_drop_dup["spell"].isin(to_del["spell"])
]
p(len(df_2400_v3_drop_dup_filtered))
to_csv_sig(df_2400_v3_drop_dup_filtered, "d:/tmp/df_2400_v3_drop_dup_filtered.csv")
l("高中排查ok")

# 2.5000篇阅读词汇排查
df_5000_word = rd_csv_sig("D:/Dropbox/06.wanjuan/99.tmp/tiku_freq_sudachipy_B.csv")
p(len(df_5000_word))
df_5000_word = df_5000_word[
    ~df_5000_word["spell"].isin(to_del["spell"])
].copy()
p(len(df_5000_word))
df_5000_word.rename(columns={"spell": "5000_spell", "count": "5000_freq"}, inplace=True)
sum_5000_word = sum(df_5000_word["5000_freq"])
print(f'词频总数：{sum_5000_word}')
df_5000_word["5000_freq_per"] = round(df_5000_word["5000_freq"] / sum_5000_word, 10)
df_5000_word.sort_values(["5000_freq"],ascending=False, inplace=True)
df_5000_word.reset_index(inplace=True, drop=True)
df_5000_word.insert(0, "5000_word_rank", range(1, len(df_5000_word) + 1))



to_csv_sig(df_5000_word, "d:/tmp/df_5000_word_filtered.csv")
l("5000阅读排查ok")


# 3. 长词汇排查
df_long_word_pkl_path = 'd:/95.pickle_files/df_long_word.pkl'

if os.path.exists(df_long_word_pkl_path):
    df_long_word = get_obj_by_pickle_path(df_long_word_pkl_path)
else:
    df_long_word = pd.read_csv(
        "d:/03.bigfile/BCCWJ_frequencylist_luw2_ver1_0.tsv",
        encoding="utf-8-sig",
        on_bad_lines="skip",
        engine="python",
        sep="\t",
    )
    export_to_pickle_path(df_long_word, df_long_word_pkl_path)
p(len(df_long_word))

pattern = r"[˚О○〇◯◎◇✖✕аА▽△▲♪☆★●■☎|。 ̊∙｡･￥\"＂〞〝＇＊＃#ⅭK₂ⅡⅢⅣ()\-－─−←（）〔〕｛｝〒〈〉／;；<=>@＜＞＠、､_【】％+＋！!&＆'%→？…·・“”」～~※℃\/「｢｣『』{}\u2460-\u2473――＝×÷（，＿:：,．\d\[\]\.\?\*a-zA-Z\uFF21-\uFF3A\uFF41-\uFF5A\u0391-\u03A9\u03B1-\u03C9]"
#pattern = r"―"
rec = re.compile(pattern)
# df_long_word = df_long_word[~df_long_word["lemma"].str.contains(cp.filter_pattern, na=False)].copy()
df_long_word = df_long_word[~df_long_word["lemma"].str.contains(rec, na=False)].copy()
p(len(df_long_word))
#to_csv_sig(df_long_word,'d:/tmp/df_long_word_tmp.csv')
df_long_word = df_long_word[
    ~df_long_word["lemma"].isin(to_del["spell"])
].copy()
p(len(df_long_word))

df_long_word.sort_values("frequency", ascending=False, inplace=True)
df_long_word=df_long_word.head(30000).copy()
df_long_word_summed = df_long_word.groupby("lemma")["frequency"].sum().reset_index()
df_long_word_summed.rename(
    columns={"lemma": "bccwj_lemma", "frequency": "bccwj_freq"}, inplace=True
)
df_long_word_summed.sort_values("bccwj_freq", ascending=False, inplace=True)
bccwj_freq_sum = df_long_word_summed["bccwj_freq"].sum()
df_long_word_summed.reset_index(inplace=True, drop=True)
df_long_word_summed.insert(0, "long_word_rank", range(1, len(df_long_word_summed) + 1))
df_long_word_summed["bccwj_freq_accu_sum"] = df_long_word_summed["bccwj_freq"].cumsum()
df_long_word_summed["bccwj_freq_per"] = round(
    df_long_word_summed["bccwj_freq"] / bccwj_freq_sum, 8
)
df_long_word_summed["bccwj_freq_accu_per"] = round(
    df_long_word_summed["bccwj_freq_accu_sum"] / bccwj_freq_sum, 8
)
# df_long_word_summed = df_long_word_summed[
#     df_long_word_summed["bccwj_freq_accu_per"] <= 0.9
# ].copy()

to_csv_sig(df_long_word_summed, "d:/Dropbox/06.wanjuan/99.tmp/df_long_tmp_v2.csv")

l("长词汇排查ok")


3407
2600
----------------------------------------高中排查ok----------------------------------------
32022
29248
词频总数：835479
----------------------------------------5000阅读排查ok----------------------------------------
841911
768406
768002
----------------------------------------长词汇排查ok----------------------------------------


In [320]:
# 4. 小学馆排查
df_base_word=rd_csv_sig(cp.w_word_raw_merged_freq_sumed_csv)
p(len(df_base_word))
pattern = cp.filter_pattern
rec = re.compile(pattern)
df_base_word = df_base_word[~df_base_word["headword"].str.contains(rec, na=False)].copy()
p(len(df_base_word))
df_base_word = df_base_word[
    ~df_base_word["headword"].isin(to_del["spell"])
].copy()
p(len(df_base_word))

# 取fused_freq>0的
df_base_word=df_base_word[df_base_word['fused_freq']>0].copy()
df_base_word=df_base_word.head(61000).copy()
p(len(df_base_word))
cols = ["word_id", "headword", "fused_freq"]
df_base_word = df_base_word[cols]
sum_base_word = sum(df_base_word["fused_freq"])
df_base_word.rename(
    columns={"headword": "base_word", "fused_freq": "base_freq"}, inplace=True
)
df_base_word["base_freq_per"] = round(df_base_word["base_freq"] / sum_base_word, 10)

df_base_word.sort_values(["base_freq"], ascending=False, inplace=True)
# df_base_word.reset_index(inplace=True,drop=True)
df_base_word.insert(0, "base_word_rank", range(1, len(df_base_word) + 1))

to_csv_sig(df_base_word,'d:/tmp/df_base_word_tmp.csv')
l("小学馆排查ok")


102513
101973
101827
61000
----------------------------------------小学馆排查ok----------------------------------------


In [321]:
# 计算总共需要多少词汇
set_long_word = set(df_long_word_summed["bccwj_lemma"])
set_base_word = set(df_base_word["base_word"])
set_5000_word = set(df_5000_word["5000_spell"])
set_all = set_long_word | set_base_word | set_5000_word
print(f"long word len: {len(set_long_word)}")
print(f"base word len: {len(set_base_word)}")
print(f"5000 kaokao len: {len(set_5000_word)}")
print(f"total len: {len(set_all)}")
l()
long_base_intersection = set_long_word & set_base_word
long_5000_intersection = set_long_word & set_5000_word
base_5000_intersection = set_base_word & set_5000_word
print(f"long_base_intersection len: {len(long_base_intersection)}")
print(f"long_5000_intersection len: {len(long_5000_intersection)}")
print(f"base_5000_intersection len: {len(base_5000_intersection)}")

l('ok')



long word len: 28995
base word len: 61000
5000 kaokao len: 29248
total len: 81642
--------------------------------------------------------------------------------
long_base_intersection len: 18137
long_5000_intersection len: 13534
base_5000_intersection len: 16997
----------------------------------------ok----------------------------------------


In [322]:
# 将4万长词汇， 6万小学馆， 3万的5000篇阅读词汇全汇总
df_all = pd.DataFrame(list(set_all), columns=["spell_all"])

df_all_m1 = pd.merge(
    df_all,
    df_long_word_summed[['long_word_rank',"bccwj_lemma", "bccwj_freq_per"]],
    left_on="spell_all",
    right_on="bccwj_lemma",
    how="left",
    indicator="ck_long_word",
)

df_all_m2 = pd.merge(
    df_all_m1,
    df_base_word,
    left_on="spell_all",
    right_on="base_word",
    how="left",
    indicator="ck_base_word",
)

df_all_m3 = pd.merge(
    df_all_m2,
    df_5000_word,
    left_on="spell_all",
    right_on="5000_spell",
    how="left",
    indicator="ck_5000_word",
)


# 筛选出高中词汇 
df_2600= df_merge_base_2400_v3[df_merge_base_2400_v3['is_for_student']==1].copy()
df_2600 = df_2600[~df_2600["spell"].isin(to_del["spell"])]
to_csv_sig(df_2600,"d:/tmp/df_2600.csv")

df_2600.rename(columns={"spell": "2600_spell"}, inplace=True)
df_all_m4 = pd.merge(
    df_all_m3,
    df_2600["2600_spell"],
    left_on="spell_all",
    right_on="2600_spell",
    how="outer",
    indicator="ck_2600_word",
)
to_csv_sig(df_all_m4,"d:/tmp/df_all_m4.csv")

l('ok')


----------------------------------------ok----------------------------------------


In [323]:
# 根据权重计算词频

df_all_m4[["bccwj_freq_per", "base_freq_per", "5000_freq_per"]] = df_all_m4[
    ["bccwj_freq_per", "base_freq_per", "5000_freq_per"]
].fillna(0)

word_5000 = 0.70
word_long = 0.20
word_base = 0.10

df_all_m4["final_freq_per"] = (
    df_all_m4["5000_freq_per"]
    * np.where(df_all_m4["ck_5000_word"] == "both", word_5000, 0)
    + df_all_m4["bccwj_freq_per"]
    * np.where(df_all_m4["ck_long_word"] == "both", word_long, 0)
    + df_all_m4["base_freq_per"]
    * np.where(df_all_m4["ck_base_word"] == "both", word_base, 0)
)

df_all_m4["final_freq_per"] = np.where(
    df_all_m4["ck_2600_word"] != "left_only",
    df_all_m4["final_freq_per"] + 0.0000492967,
    df_all_m4["final_freq_per"],
)

def cal_freq(freq):
    if freq <= 8000:
        return freq
    elif freq <= 12000:
        return round(8000 + (freq - 8000) / 2, 2)
    elif freq <= 20000:
        return round(10000 + (freq - 12000) / 4, 2)
    elif freq <= 30000:
        return round(12000 + (freq - 20000) / 8, 2)
    elif freq <= 110000:
        return round(13250 + (freq - 30000) / 16, 2)
    else:
        return round(18250 + (freq - 110000) / 100, 2)




df_all_m4_new=df_all_m4.sort_values(["final_freq_per"], ascending=False)
df_all_m4_new.reset_index(inplace=True, drop=True)
df_all_m4_new.insert(0, "final_freq", range(1, len(df_all_m4_new) + 1))
df_all_m4_new["final_freq"] = df_all_m4_new["final_freq"].apply(cal_freq)

to_csv_sig(df_all_m4_new,"d:/tmp/df_all_m4_new.csv")
l('ok')


----------------------------------------ok----------------------------------------


In [324]:
# 和老的词频进行关联， 找到需要删除的word_id， 同时更新词频
# 把2600词汇spell填充到df_merged_freq的 spell_all字段里
df_all_m4_new["spell_all"] = df_all_m4_new["spell_all"].fillna(
    df_all_m4_new["2600_spell"]
)
df_all_m4_new["is_for_student"] = np.where(
    df_all_m4_new["ck_2600_word"].isin(["both", "right_only"]), 1, 0
)

to_csv_sig(df_all_m4_new, "d:/tmp/df_all_m4_new.csv")


df_base_word_130000_to_del = df_base_word_130000[
    ~df_base_word_130000["spell"].isin(df_all_m4_new["spell_all"])
]
df_base_word_remain = df_base_word_130000[
    df_base_word_130000["spell"].isin(df_all_m4_new["spell_all"])
].copy()

freq_mapping = df_all_m4_new.set_index("spell_all")["final_freq"]
is_student_mapping = df_all_m4_new.set_index('spell_all')['is_for_student']

df_base_word_remain.loc[:, "frequency"] = (
    df_base_word_remain["spell"]
    .map(freq_mapping)
    .combine_first(df_base_word_remain["frequency"])
)
df_base_word_remain.loc[:, "spell_type"] = (
    df_base_word_remain["spell"]
    .map(is_student_mapping)
    .combine_first(df_base_word_remain["spell_type"])
)
df_base_word_remain.sort_values("frequency", ascending=True, inplace=True)

to_csv_sig(df_base_word_130000_to_del, "d:/tmp/df_base_word_130000_to_del.csv")
to_csv_sig(df_base_word_remain, "d:/tmp/df_base_word_remain.csv")

l("ok")

----------------------------------------ok----------------------------------------
